In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import random
import seaborn as sn
import datetime as dt
import re
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.utils import resample
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from gensim.models import Word2Vec, KeyedVectors
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, SCORERS, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import Lasso, LassoLars, LinearRegression, Ridge
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
%matplotlib inline

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train = pd.read_csv('/Users/skylark/Desktop/healthcare II/train.csv')
test = pd.read_csv('/Users/skylark/Desktop/healthcare II/test.csv')
sample = pd.read_csv('/Users/skylark/Desktop/healthcare II/sample_submission_lfbv3c3.csv')
train_data_dict = pd.read_csv('/Users/skylark/Desktop/healthcare II/train_data_dict.csv')

In [3]:
data = pd.concat([train, test]).reset_index(drop=True)
data_new = data.copy()
data_new.isnull().sum()

case_id                                   0
Hospital_code                             0
Hospital_type_code                        0
City_Code_Hospital                        0
Hospital_region_code                      0
Available Extra Rooms in Hospital         0
Department                                0
Ward_Type                                 0
Ward_Facility_Code                        0
Bed Grade                               148
patientid                                 0
City_Code_Patient                      6689
Type of Admission                         0
Severity of Illness                       0
Visitors with Patient                     0
Age                                       0
Admission_Deposit                         0
Stay                                 137057
dtype: int64

In [4]:
data_new['Bed Grade'].fillna(2, inplace=True)
data_new.City_Code_Patient.fillna(8, inplace=True)

In [5]:
cat_cols = ['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age']
for col in cat_cols:
    col_dict = {}
    for i,j in enumerate(sorted(data_new[col].dropna().unique())):
        col_dict[j] = i
    data_new[col] = data_new[col].map(lambda x: col_dict[x])
    
stay_dict = {}
rev_stay_dict = {}
for i,j in enumerate(sorted(train.dropna().Stay.unique())):
    stay_dict[j] = i
    rev_stay_dict[i] = j
    
target = data.Stay.dropna()
target = target.map(lambda x: stay_dict[x])
data_new.drop('Stay', axis=1, inplace=True)
#data_new['patient_count'] = data_new.patientid.map(data_new.patientid.value_counts())    

In [115]:
patient = pd.read_csv('/Users/skylark/Desktop/healthcare II/patient.csv').iloc[:,1:]
hospital = pd.read_csv('/Users/skylark/Desktop/healthcare II/hospital.csv').iloc[:,1:]
testing = pd.read_csv('/Users/skylark/Desktop/healthcare II/final_data.csv').iloc[:,1:]

### Hospital

In [6]:
hospital = data_new.iloc[:,1:11]

In [7]:
hospital['new_hosp_code'] = 10000*hospital.City_Code_Hospital + 1000*hospital.Hospital_region_code + 100*hospital.Hospital_type_code + hospital.Hospital_code
hospital['new_hosp_dept_code'] = hospital.new_hosp_code*10 + hospital.Department
hospital['dept_ward_code'] = hospital.new_hosp_dept_code*10 + hospital.Ward_Type

In [8]:
pat_recur_hosp = pd.DataFrame(hospital.groupby('new_hosp_code')['patientid'].value_counts(normalize=True)*1000).rename(columns={'patientid': 'patient_recur_count'}).reset_index()

In [9]:
hospital = hospital.merge(pat_recur_hosp, how='left', left_on=['new_hosp_code', 'patientid'], right_on=['new_hosp_code', 'patientid'])
hospital['hosp_visited'] = hospital.new_hosp_code.map(hospital.new_hosp_code.value_counts(normalize=True))
hospital = hospital.merge(pd.DataFrame(hospital.groupby('new_hosp_code')['new_hosp_dept_code'].value_counts(normalize=True).sort_index()).rename(columns={'new_hosp_dept_code': 'hosp_dept_visited'}).reset_index(), how='left', left_on=['new_hosp_code','new_hosp_dept_code'], right_on=['new_hosp_code','new_hosp_dept_code'])
hospital = hospital.merge(pd.DataFrame(hospital.groupby('new_hosp_code')['dept_ward_code'].value_counts(normalize=True)).rename(columns={'dept_ward_code': 'hosp_ward_visited'}).reset_index(), how='left', left_on = ['new_hosp_code','dept_ward_code'], right_on = ['new_hosp_code','dept_ward_code'])

In [10]:
hospital['city_visited'] = hospital.City_Code_Hospital.map(hospital.City_Code_Hospital.value_counts(normalize=True))
hospital['region_visited'] = hospital.Hospital_region_code.map(hospital.Hospital_region_code.value_counts(normalize=True))
hospital['type_hosp_visited'] = hospital.Hospital_type_code.map(hospital.Hospital_type_code.value_counts(normalize=True))
hospital

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,new_hosp_code,new_hosp_dept_code,dept_ward_code,patient_recur_count,hosp_visited,hosp_dept_visited,hosp_ward_visited,city_visited,region_visited,type_hosp_visited
0,8,2,3,2,3,3,2,5,2.0,31397,32208,322083,3220832,0.189251,0.011601,0.064913,0.037472,0.099988,0.197453,0.145220
1,2,2,5,2,2,3,3,5,2.0,31397,52202,522023,5220233,0.137419,0.015976,0.090422,0.080940,0.097465,0.197453,0.145220
2,10,4,1,0,2,1,3,4,2.0,31397,10410,104101,1041013,0.147732,0.029722,0.082878,0.045058,0.173565,0.418993,0.077779
3,26,1,2,1,2,3,2,3,2.0,31397,21126,211263,2112632,0.063127,0.104333,0.105296,0.044905,0.163146,0.383554,0.217091
4,26,1,2,1,2,3,3,3,2.0,31397,21126,211263,2112633,0.063127,0.104333,0.105296,0.025546,0.163146,0.383554,0.217091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455490,11,1,2,1,4,1,1,3,3.0,41160,21111,211111,2111111,0.040279,0.054506,0.108310,0.095541,0.163146,0.383554,0.217091
455491,25,4,1,0,2,3,2,4,4.0,30985,10425,104253,1042532,0.071195,0.030837,0.073686,0.018867,0.173565,0.418993,0.077779
455492,30,2,3,2,2,1,2,0,4.0,81811,32230,322301,3223012,0.138600,0.015840,0.103396,0.049480,0.099988,0.197453,0.145220
455493,5,0,1,0,2,1,2,4,4.0,57021,10005,100051,1000512,0.134264,0.016351,0.141783,0.102846,0.173565,0.418993,0.449467


In [11]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('Hospital_type_code')['patientid'].value_counts(normalize=True)*100).rename(columns={'patientid':'hosp_unique_type'}).reset_index(), how='left', left_on=['Hospital_type_code','patientid'], right_on=['Hospital_type_code','patientid'])

In [12]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('City_Code_Hospital')['patientid'].value_counts(normalize=True)*100).rename(columns={'patientid':'city_pat_visited'}).reset_index(), how='left', left_on=['City_Code_Hospital','patientid'], right_on=['City_Code_Hospital','patientid'])

In [13]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('Hospital_region_code')['patientid'].value_counts(normalize=True)*1000).rename(columns={'patientid':'region_pat_visited'}).reset_index(), how='left', left_on=['Hospital_region_code','patientid'], right_on=['Hospital_region_code','patientid'])

In [14]:
hospital['percent_extra_room'] = hospital.new_hosp_code.map((hospital.groupby('new_hosp_code')['Available Extra Rooms in Hospital'].sum()/(hospital.groupby('new_hosp_code')['Available Extra Rooms in Hospital'].sum()).sum())*100)
hospital['percent_extra_room_dept'] = hospital.new_hosp_dept_code.map((hospital.groupby('new_hosp_dept_code')['Available Extra Rooms in Hospital'].sum()/((hospital.groupby('new_hosp_dept_code')['Available Extra Rooms in Hospital'].sum()).sum()))*100)
hospital

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,...,hosp_dept_visited,hosp_ward_visited,city_visited,region_visited,type_hosp_visited,hosp_unique_type,city_pat_visited,region_pat_visited,percent_extra_room,percent_extra_room_dept
0,8,2,3,2,3,3,2,5,2.0,31397,...,0.064913,0.037472,0.099988,0.197453,0.145220,0.004535,0.004391,0.044475,1.249669,0.077757
1,2,2,5,2,2,3,3,5,2.0,31397,...,0.090422,0.080940,0.097465,0.197453,0.145220,0.004535,0.004505,0.044475,1.340682,0.115604
2,10,4,1,0,2,1,3,4,2.0,31397,...,0.082878,0.045058,0.173565,0.418993,0.077779,0.005645,0.002530,0.020959,2.871772,0.195353
3,26,1,2,1,2,3,2,3,2.0,31397,...,0.105296,0.044905,0.163146,0.383554,0.217091,0.003034,0.004037,0.034343,10.752384,1.063383
4,26,1,2,1,2,3,3,3,2.0,31397,...,0.105296,0.025546,0.163146,0.383554,0.217091,0.003034,0.004037,0.034343,10.752384,1.063383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455490,11,1,2,1,4,1,1,3,3.0,41160,...,0.108310,0.095541,0.163146,0.383554,0.217091,0.001011,0.001346,0.005724,5.873092,0.567375
455491,25,4,1,0,2,3,2,4,4.0,30985,...,0.073686,0.018867,0.173565,0.418993,0.077779,0.002823,0.001265,0.005240,2.811188,0.197276
455492,30,2,3,2,2,1,2,0,4.0,81811,...,0.103396,0.049480,0.099988,0.197453,0.145220,0.001512,0.002196,0.011119,1.708309,0.139989
455493,5,0,1,0,2,1,2,4,4.0,57021,...,0.141783,0.102846,0.173565,0.418993,0.449467,0.000488,0.001265,0.005240,1.682550,0.195353


In [15]:
unique_wards = {}
for code in hospital.new_hosp_code.unique():
    unique_wards[code] = len(hospital[hospital.new_hosp_code==code]['Ward_Type'].unique())
    
unique_wards_dept = {}
for code in hospital.new_hosp_dept_code.unique():
    unique_wards_dept[code] = len(hospital[hospital.new_hosp_dept_code==code]['Ward_Type'].unique())    

In [16]:
hospital['unique_wards_hosp'] = hospital.new_hosp_code.map(unique_wards)
hospital['unique_wards_dept'] = hospital.new_hosp_dept_code.map(unique_wards_dept)

In [17]:
hospital['hosp_ward_facility'] = hospital.new_hosp_dept_code*10 + hospital.Ward_Facility_Code

In [18]:
hospital['dept_ward_ward_fac'] = hospital.dept_ward_code*10 + hospital.Ward_Facility_Code
hospital['ward_facility_hosp_percent'] = hospital.dept_ward_ward_fac.map(hospital.dept_ward_ward_fac.value_counts(normalize=True)*100)

In [19]:
hospital.iloc[:,9:]

,patientid,new_hosp_code,new_hosp_dept_code,dept_ward_code,patient_recur_count,hosp_visited,hosp_dept_visited,hosp_ward_visited,city_visited,region_visited,...,hosp_unique_type,city_pat_visited,region_pat_visited,percent_extra_room,percent_extra_room_dept,unique_wards_hosp,unique_wards_dept,hosp_ward_facility,dept_ward_ward_fac,ward_facility_hosp_percent
0,31397,32208,322083,3220832,0.189251,0.011601,0.064913,0.037472,0.099988,0.197453,...,0.004535,0.004391,0.044475,1.249669,0.077757,4,2,3220835,32208325,0.043469
1,31397,52202,522023,5220233,0.137419,0.015976,0.090422,0.080940,0.097465,0.197453,...,0.004535,0.004505,0.044475,1.340682,0.115604,2,2,5220235,52202335,0.129310
2,31397,10410,104101,1041013,0.147732,0.029722,0.082878,0.045058,0.173565,0.418993,...,0.005645,0.002530,0.020959,2.871772,0.195353,3,3,1041014,10410134,0.133920
3,31397,21126,211263,2112632,0.063127,0.104333,0.105296,0.044905,0.163146,0.383554,...,0.003034,0.004037,0.034343,10.752384,1.063383,4,4,2112633,21126323,0.468501
4,31397,21126,211263,2112633,0.063127,0.104333,0.105296,0.025546,0.163146,0.383554,...,0.003034,0.004037,0.034343,10.752384,1.063383,4,4,2112633,21126333,0.266523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455490,41160,21111,211111,2111111,0.040279,0.054506,0.108310,0.095541,0.163146,0.383554,...,0.001011,0.001346,0.005724,5.873092,0.567375,2,2,2111113,21111113,0.520752
455491,30985,10425,104253,1042532,0.071195,0.030837,0.073686,0.018867,0.173565,0.418993,...,0.002823,0.001265,0.005240,2.811188,0.197276,3,3,1042534,10425324,0.058178
455492,81811,32230,322301,3223012,0.138600,0.015840,0.103396,0.049480,0.099988,0.197453,...,0.001512,0.002196,0.011119,1.708309,0.139989,3,3,3223010,32230120,0.078376
455493,57021,10005,100051,1000512,0.134264,0.016351,0.141783,0.102846,0.173565,0.418993,...,0.000488,0.001265,0.005240,1.682550,0.195353,2,2,1000514,10005124,0.168169


In [20]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('dept_ward_code')['Available Extra Rooms in Hospital'].sum()).rename(columns={'Available Extra Rooms in Hospital':'extra_rooms_ward'}).reset_index(), how='left', left_on='dept_ward_code', right_on='dept_ward_code')

In [21]:
hospital['dept_ward_bed_grade'] = hospital.dept_ward_code*10 + hospital['Bed Grade']
hospital['dept_ward_bed_grad'] = hospital.dept_ward_bed_grade.map(hospital.dept_ward_bed_grade.value_counts(normalize=True)*100)

In [22]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('new_hosp_dept_code')['Bed Grade'].value_counts()).rename(columns={'Bed Grade':'dept_bed_grade_count'}).reset_index(), how='left', left_on=['new_hosp_dept_code','Bed Grade'], right_on=['new_hosp_dept_code','Bed Grade'])

In [23]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('new_hosp_code')['Bed Grade'].value_counts(normalize=True)).rename(columns={'Bed Grade':'hosp_bed_grade_count'}).reset_index(), how='left', left_on=['new_hosp_code','Bed Grade'], right_on=['new_hosp_code','Bed Grade'])

In [24]:
hospital = hospital.merge(hospital.groupby('new_hosp_dept_code')['Available Extra Rooms in Hospital'].sum().reset_index().rename(columns={'Available Extra Rooms in Hospital':'hosp_dept_extra_rooms'}), how='left', left_on='new_hosp_dept_code', right_on='new_hosp_dept_code')

In [25]:
hospital = hospital.merge(hospital.groupby('Department')['Available Extra Rooms in Hospital'].sum().reset_index().rename(columns={'Available Extra Rooms in Hospital':'extra_rooms_dept'}), how='left', left_on='Department', right_on='Department')

In [26]:
hospital['hosp_dept_extra_rooms'] = hospital.hosp_dept_extra_rooms/hospital.extra_rooms_dept

In [27]:
hospital.drop('extra_rooms_dept', axis=1, inplace=True)

In [28]:
hospital = hospital[['new_hosp_code', 'new_hosp_dept_code', 'dept_ward_code',
       'patient_recur_count', 'hosp_visited', 'hosp_dept_visited',
       'hosp_ward_visited', 'city_visited', 'region_visited',
       'type_hosp_visited', 'hosp_unique_type', 'city_pat_visited',
       'region_pat_visited', 'percent_extra_room', 'percent_extra_room_dept',
       'unique_wards_hosp', 'unique_wards_dept', 'hosp_ward_facility',
       'dept_ward_ward_fac', 'ward_facility_hosp_percent', 'extra_rooms_ward',
       'dept_ward_bed_grade', 'dept_ward_bed_grad', 'dept_bed_grade_count',
       'hosp_bed_grade_count', 'hosp_dept_extra_rooms']]

In [29]:
hospital = pd.concat([data_new[['Type of Admission','Severity of Illness','Admission_Deposit']], hospital], axis=1)

In [30]:
pd.DataFrame(hospital.groupby('new_hosp_code')['Type of Admission'].value_counts()).rename(columns={'Type of Admission':'hosp_admission_type_count'}).reset_index()

,new_hosp_code,Type of Admission,hosp_admission_type_count
0,10005,1,4296
1,10005,0,2385
2,10005,2,767
3,10014,1,12720
4,10014,0,8804
...,...,...,...
90,110128,0,6692
91,110128,2,4164
92,131318,1,2661
93,131318,2,1473


In [31]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('new_hosp_code')['Type of Admission'].value_counts()).rename(columns={'Type of Admission':'hosp_admission_type_count'}).reset_index(), how='left', left_on=['new_hosp_code','Type of Admission'], right_on=['new_hosp_code','Type of Admission'])

In [32]:
hospital['hosp_admission_type_count'] = hospital.hosp_admission_type_count/hospital.new_hosp_code.map(hospital.new_hosp_code.value_counts())

In [33]:
hospital['hospital_admission_code'] = hospital.new_hosp_code*10 + hospital['Type of Admission']
hospital['hosp_illness_code'] = hospital.new_hosp_code*10 + hospital['Severity of Illness']
hospital = hospital.merge(pd.DataFrame(hospital.groupby('new_hosp_code')['Severity of Illness'].value_counts()).rename(columns={'Severity of Illness':'hosp_illness_type'}).reset_index(), how='left', left_on=['new_hosp_code','Severity of Illness'], right_on=['new_hosp_code','Severity of Illness'])
hospital['hosp_illness_type'] = hospital['hosp_illness_type']/hospital.new_hosp_code.map(hospital.new_hosp_code.value_counts())

In [34]:
hospital['mean_deposit_hosp'] = hospital.new_hosp_code.map(hospital.groupby('new_hosp_code')['Admission_Deposit'].mean())

In [35]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('new_hosp_dept_code')['Type of Admission'].value_counts()).rename(columns={'Type of Admission':'admission_type_dept_count'}).reset_index(), how='left', left_on=['new_hosp_dept_code','Type of Admission'], right_on=['new_hosp_dept_code','Type of Admission'])
hospital['admission_type_dept_count'] = hospital['admission_type_dept_count']/hospital.new_hosp_dept_code.map(hospital.new_hosp_dept_code.value_counts())

In [36]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('new_hosp_dept_code')['Severity of Illness'].value_counts()).rename(columns={'Severity of Illness':'illness_type_dept_count'}).reset_index(), how='left', left_on=['new_hosp_dept_code','Severity of Illness'], right_on=['new_hosp_dept_code','Severity of Illness'])
hospital['illness_type_dept_count'] = hospital['illness_type_dept_count']/hospital.new_hosp_dept_code.map(hospital.new_hosp_dept_code.value_counts())

In [37]:
hospital['hosp_dept_deposit_mean'] = hospital.new_hosp_dept_code.map(hospital.groupby('new_hosp_dept_code')['Admission_Deposit'].mean())
hospital

,Type of Admission,Severity of Illness,Admission_Deposit,new_hosp_code,new_hosp_dept_code,dept_ward_code,patient_recur_count,hosp_visited,hosp_dept_visited,hosp_ward_visited,...,hosp_bed_grade_count,hosp_dept_extra_rooms,hosp_admission_type_count,hospital_admission_code,hosp_illness_code,hosp_illness_type,mean_deposit_hosp,admission_type_dept_count,illness_type_dept_count,hosp_dept_deposit_mean
0,0,0,4911.0,32208,322083,3220832,0.189251,0.011601,0.064913,0.037472,...,0.432438,0.009088,0.357873,322080,322080,0.167108,4920.411809,0.355685,0.206997,4959.915452
1,1,0,5954.0,52202,522023,5220233,0.137419,0.015976,0.090422,0.080940,...,0.385873,0.013511,0.508589,522021,522020,0.184142,4998.556685,0.460486,0.193009,5017.291793
2,1,0,4745.0,10410,104101,1041013,0.147732,0.029722,0.082878,0.045058,...,0.390383,0.023814,0.495863,104101,104100,0.180455,4556.475772,0.646168,0.188948,4894.282531
3,1,0,7272.0,21126,211263,2112632,0.063127,0.104333,0.105296,0.044905,...,0.408939,0.124281,0.504977,211261,211260,0.202113,4895.473644,0.496403,0.205635,4971.854117
4,1,0,5558.0,21126,211263,2112633,0.063127,0.104333,0.105296,0.025546,...,0.408939,0.124281,0.504977,211261,211260,0.202113,4895.473644,0.496403,0.205635,4971.854117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455490,0,1,6313.0,21111,211111,2111111,0.040279,0.054506,0.108310,0.095541,...,0.363516,0.069163,0.365892,211110,211111,0.321948,4925.715672,0.226106,0.258089,5507.190405
455491,0,2,3510.0,10425,104253,1042532,0.071195,0.030837,0.073686,0.018867,...,0.221487,0.023056,0.353125,104250,104252,0.542859,4734.464901,0.412560,0.556522,4849.439614
455492,2,1,7190.0,32230,322301,3223012,0.138600,0.015840,0.103396,0.049480,...,0.163132,0.017065,0.139848,322302,322301,0.263756,5305.495773,0.131367,0.305630,6151.651475
455493,1,1,5435.0,10005,100051,1000512,0.134264,0.016351,0.141783,0.102846,...,0.208244,0.023814,0.576799,100051,100051,0.350295,4925.916890,0.709280,0.294508,5399.132576


In [38]:
hospital=hospital.merge(pd.DataFrame(hospital.groupby('dept_ward_code')['Type of Admission'].value_counts()).rename(columns={'Type of Admission':'admission_hosp_dept_ward_count'}).reset_index(), how='left', left_on=['dept_ward_code','Type of Admission'], right_on=['dept_ward_code','Type of Admission'])
hospital['admission_hosp_dept_ward_count'] = hospital['admission_hosp_dept_ward_count']/hospital.dept_ward_code.map(hospital.dept_ward_code.value_counts())

In [40]:
hospital=hospital.merge(pd.DataFrame(hospital.groupby('dept_ward_code')['Severity of Illness'].value_counts()).rename(columns={'Severity of Illness':'illness_hosp_dept_ward_count'}).reset_index(), how='left', left_on=['dept_ward_code','Severity of Illness'], right_on=['dept_ward_code','Severity of Illness'])
hospital['illness_hosp_dept_ward_count'] = hospital['illness_hosp_dept_ward_count']/hospital.dept_ward_code.map(hospital.dept_ward_code.value_counts())

In [41]:
hospital['dept_ward_mean_deposit'] = hospital.dept_ward_code.map(hospital.groupby('dept_ward_code')['Admission_Deposit'].mean())

In [42]:
hospital.drop(['hospital_admission_code','hosp_illness_code'], axis=1, inplace=True)

In [43]:
pd.DataFrame(hospital.groupby('hosp_ward_facility')['Type of Admission'].value_counts()).rename(columns={'Type of Admission':'admission_ward_fac_count'}).reset_index()

,hosp_ward_facility,Type of Admission,admission_ward_fac_count
0,1000504,1,127
1,1000504,0,50
2,1000504,2,21
3,1000514,1,749
4,1000514,0,175
...,...,...,...
429,13131821,0,947
430,13131831,1,224
431,13131831,2,96
432,13131831,0,74


In [44]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('hosp_ward_facility')['Type of Admission'].value_counts()).rename(columns={'Type of Admission':'admission_ward_fac_count'}).reset_index(), how='left', left_on=['hosp_ward_facility','Type of Admission'], right_on=['hosp_ward_facility','Type of Admission'])
hospital['admission_ward_fac_count'] = hospital['admission_ward_fac_count']/hospital.hosp_ward_facility.map(hospital.hosp_ward_facility.value_counts())

In [45]:
hospital = hospital.merge(pd.DataFrame(hospital.groupby('hosp_ward_facility')['Severity of Illness'].value_counts()).rename(columns={'Severity of Illness':'illness_ward_fac_count'}).reset_index(), how='left', left_on=['hosp_ward_facility','Severity of Illness'], right_on=['hosp_ward_facility','Severity of Illness'])
hospital['illness_ward_fac_count'] = hospital['illness_ward_fac_count']/hospital.hosp_ward_facility.map(hospital.hosp_ward_facility.value_counts())

In [46]:
hospital = hospital.iloc[:,3:]

### Patient

In [115]:
patient = data_new[['patientid', 'City_Code_Patient', 'Type of Admission','Severity of Illness','Available Extra Rooms in Hospital', 'Visitors with Patient', 'Age','Admission_Deposit','Hospital_code', 'Hospital_type_code', 'City_Code_Hospital','Hospital_region_code','Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade']]

In [116]:
patient['patient_count'] = patient.patientid.map(patient.patientid.value_counts())

In [49]:
pat_city_unique = {}
for pat in patient.patientid.unique():
    pat_city_unique[pat] = len(patient[patient.patientid==pat]['City_Code_Hospital'].unique())

In [50]:
pat_hosp_unique = {}
for pat in patient.patientid.unique():
    pat_hosp_unique[pat] = len(patient[patient.patientid==pat]['Hospital_code'].unique())

In [51]:
pat_hosp_type_unique = {}
for pat in patient.patientid.unique():
    pat_hosp_type_unique[pat] = len(patient[patient.patientid==pat]['Hospital_type_code'].unique())

In [52]:
pat_region_unique = {}
for pat in patient.patientid.unique():
    pat_region_unique[pat] = len(patient[patient.patientid==pat]['Hospital_region_code'].unique())

In [53]:
pat_dept_unique = {}
for pat in patient.patientid.unique():
    pat_dept_unique[pat] = len(patient[patient.patientid==pat]['Department'].unique())

In [54]:
pat_ward_type_unique = {}
for pat in patient.patientid.unique():
    pat_ward_type_unique[pat] = len(patient[patient.patientid==pat]['Ward_Type'].unique())

In [55]:
pat_ward_fac_unique = {}
for pat in patient.patientid.unique():
    pat_ward_fac_unique[pat] = len(patient[patient.patientid==pat]['Ward_Facility_Code'].unique())

In [117]:
patient['pat_unique_city'] = patient.patientid.map(pat_city_unique)
patient['pat_unique_region'] = patient.patientid.map(pat_region_unique)
patient['pat_unique_hosp_type'] = patient.patientid.map(pat_hosp_type_unique)
patient['pat_unique_hosp'] = patient.patientid.map(pat_hosp_unique)
patient['pat_dept_type'] = patient.patientid.map(pat_dept_unique)
patient['pat_unique_ward_type'] = patient.patientid.map(pat_ward_type_unique)
patient['pat_unique_ward_fac'] = patient.patientid.map(pat_ward_fac_unique)

In [118]:
patient = patient.merge(pd.DataFrame(patient.groupby('patientid')['Type of Admission'].value_counts()).rename(columns = {'Type of Admission':'pat_admission_type_count'}).reset_index(), how='left', left_on=['patientid','Type of Admission'], right_on=['patientid','Type of Admission'])
patient['pat_admission_type_count'] = patient['pat_admission_type_count']/patient.patientid.map(patient.patientid.value_counts())

In [119]:
patient = patient.merge(pd.DataFrame(patient.groupby('patientid')['Severity of Illness'].value_counts()).rename(columns = {'Severity of Illness':'pat_illness_type_count'}).reset_index(), how='left', left_on=['patientid','Severity of Illness'], right_on=['patientid','Severity of Illness'])
patient['pat_illness_type_count'] = patient['pat_illness_type_count']/patient.patientid.map(patient.patientid.value_counts())

In [59]:
pat_city_outside = {}
for pat in patient.patientid.unique():
    pat_city = patient[patient.patientid==pat]['City_Code_Patient'].unique()
    out_of_town = patient[patient.patientid==pat]['City_Code_Hospital'].unique()
    pat_city_outside[pat] = len(set(out_of_town) - set(pat_city))

In [120]:
patient['outside_city_visited'] = patient.patientid.map(pat_city_outside)

In [121]:
patient['pat_visitors_code'] = patient.patientid*10 + patient['Visitors with Patient']

In [122]:
patient['pat_visitors_count'] = patient.pat_visitors_code.map((patient.pat_visitors_code.value_counts(normalize=True)*1000))

In [123]:
patient['pat_bed_grade_code'] = patient.patientid*10 + patient['Bed Grade']
patient['pat_bed_grade_count'] = patient.pat_bed_grade_code.map(patient.pat_bed_grade_code.value_counts())

In [124]:
patient['pat_age_code'] = patient.patientid*10 + patient.Age

In [125]:
patient['pat_extra_rooms_code'] = (patient.patientid*100 + patient['Available Extra Rooms in Hospital'])
patient['pat_extra_rooms_count'] = (patient.patientid*100 + patient['Available Extra Rooms in Hospital']).map((patient.patientid*100 + patient['Available Extra Rooms in Hospital']).value_counts())

In [126]:
patient['pat_admission_code'] = patient.patientid*10 + patient['Type of Admission']
patient['pat_illness_code'] = patient.patientid*10 + patient['Severity of Illness']

In [127]:
patient['pat_mean_deposit'] = patient.patientid.map(patient.groupby('patientid')['Admission_Deposit'].mean())
patient['pat_visitors_mean_deposit'] = patient.pat_visitors_code.map(patient.groupby('pat_visitors_code')['Admission_Deposit'].mean())
patient['pat_bed_grade_mean_deposit'] = patient.pat_bed_grade_code.map(patient.groupby('pat_bed_grade_code')['Admission_Deposit'].mean())
patient['pat_extra_rooms_mean_deposit'] = patient.pat_extra_rooms_code.map(patient.groupby('pat_extra_rooms_code')['Admission_Deposit'].mean())
patient['pat_admission_mean_deposit'] = patient.pat_admission_code.map(patient.groupby('pat_admission_code')['Admission_Deposit'].mean())

In [128]:
patient['pat_id_admission_type'] = patient.patientid*patient['Type of Admission']
patient['pat_id_illness_type'] = patient.patientid*patient['Severity of Illness']

In [142]:
patient['pat_illness_mean_deposit'] = patient.pat_illness_code.map(patient.groupby('pat_illness_code')['Admission_Deposit'].mean())

In [144]:
patient.iloc[:,-20:]

,pat_illness_type_count,outside_city_visited,pat_visitors_code,pat_visitors_count,pat_bed_grade_code,pat_bed_grade_count,pat_age_code,pat_extra_rooms_code,pat_extra_rooms_count,pat_admission_code,pat_illness_code,pat_mean_deposit,pat_visitors_mean_deposit,pat_bed_grade_mean_deposit,pat_extra_rooms_mean_deposit,pat_admission_mean_deposit,pat_id_admission_type,pat_id_illness_type,Admission_Deposit,pat_illness_mean_deposit
0,1.0,7,313972,0.030736,313972.0,7,313975,3139703,2,313970,313970,5951.0,5951.0,5526.571429,6694.000000,5406.333333,0,0,4911.0,5951.0
1,1.0,7,313972,0.030736,313972.0,7,313975,3139702,9,313971,313970,5951.0,5951.0,5526.571429,5799.444444,5853.500000,31397,0,5954.0,5951.0
2,1.0,7,313972,0.030736,313972.0,7,313975,3139702,9,313971,313970,5951.0,5951.0,5526.571429,5799.444444,5853.500000,31397,0,4745.0,5951.0
3,1.0,7,313972,0.030736,313972.0,7,313975,3139702,9,313971,313970,5951.0,5951.0,5526.571429,5799.444444,5853.500000,31397,0,7272.0,5951.0
4,1.0,7,313972,0.030736,313972.0,7,313975,3139702,9,313971,313970,5951.0,5951.0,5526.571429,5799.444444,5853.500000,31397,0,5558.0,5951.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455490,1.0,1,411604,0.002195,411603.0,1,411604,4116004,1,411600,411601,6313.0,6313.0,6313.000000,6313.000000,6313.000000,0,41160,6313.0,6313.0
455491,1.0,1,309852,0.002195,309854.0,1,309850,3098502,1,309850,309852,3510.0,3510.0,3510.000000,3510.000000,3510.000000,0,61970,3510.0,3510.0
455492,1.0,1,818112,0.002195,818114.0,1,818110,8181102,1,818112,818111,7190.0,7190.0,7190.000000,7190.000000,7190.000000,163622,81811,7190.0,7190.0
455493,1.0,1,570212,0.002195,570214.0,1,570214,5702102,1,570211,570211,5435.0,5435.0,5435.000000,5435.000000,5435.000000,57021,57021,5435.0,5435.0


In [146]:
patient['pat_age_mean_deposit'] = patient.pat_age_code.map(patient.groupby('pat_age_code')['Admission_Deposit'].mean())

In [150]:
patient = patient.iloc[:,16:]

In [15]:
data_new = data_new.merge(pd.DataFrame(data_new.groupby('patientid')['Department'].value_counts()).rename(columns={'Department':'pat_dept_visited'}).reset_index(), how='left', left_on=['patientid', 'Department'], right_on=['patientid', 'Department'])
data_new['pat_dept_visited'] = data_new['pat_dept_visited']/data_new.patientid.map(data_new.patientid.value_counts())

In [16]:
data_new = data_new.merge(pd.DataFrame(data_new.groupby('patientid')['Ward_Type'].value_counts()).rename(columns={'Ward_Type':'pat_ward_visited'}).reset_index(), how='left', left_on = ['patientid','Ward_Type'], right_on=['patientid','Ward_Type'])
data_new['pat_ward_visited'] = data_new['pat_ward_visited']/data_new.patientid.map(data_new.patientid.value_counts())

In [17]:
data_new['pat_count'] = data_new.patientid.map(data_new.patientid.value_counts())

In [18]:
data_new = data_new.merge(pd.DataFrame(data_new.groupby('patientid')['Hospital_code'].value_counts()).rename(columns={'Hospital_code':'pat_hosp_visited'}).reset_index(), how='left', left_on=['patientid','Hospital_code'], right_on=['patientid','Hospital_code'])
data_new['pat_hosp_visited'] = data_new['pat_hosp_visited']/data_new.pat_count

In [19]:
data_new = data_new.merge(pd.DataFrame(data_new.groupby('patientid')['Ward_Facility_Code'].value_counts()).rename(columns = {'Ward_Facility_Code':'pat_ward_fac_count'}).reset_index(), how='left', left_on=['patientid','Ward_Facility_Code'], right_on=['patientid','Ward_Facility_Code'])
data_new['pat_ward_fac_count'] = data_new['pat_ward_fac_count']/data_new.pat_count

In [23]:
patient = pd.concat([data_new.iloc[:,-5:], patient],axis=1)

In [35]:
patient = pd.concat([data_new.loc[:,['Hospital_type_code','City_Code_Hospital','Hospital_region_code','patientid']],patient], axis=1)

In [44]:
patient = patient.merge(pd.DataFrame(patient.groupby('patientid')['Hospital_type_code'].value_counts()).rename(columns={'Hospital_type_code':'hosp_type_count'}).reset_index(), how='left', left_on=['patientid','Hospital_type_code'], right_on=['patientid','Hospital_type_code'])
patient['hosp_type_count'] = patient['hosp_type_count']/patient.pat_count

In [60]:
patient = patient.merge(pd.DataFrame(patient.groupby('patientid')['City_Code_Hospital'].value_counts()).rename(columns={'City_Code_Hospital':'pat_cities_visited_count'}).reset_index(), how='left', left_on=['patientid','City_Code_Hospital'], right_on=['patientid','City_Code_Hospital'])
patient['pat_cities_visited_count'] = patient['pat_cities_visited_count']/patient.pat_count

In [62]:
patient = patient.merge(pd.DataFrame(patient.groupby('patientid')['Hospital_region_code'].value_counts()).rename(columns={'Hospital_region_code':'pat_regions_visited_count'}).reset_index(), how='left', left_on=['patientid','Hospital_region_code'], right_on=['patientid','Hospital_region_code'])
patient['pat_regions_visited_count'] = patient['pat_regions_visited_count']/patient.pat_count

In [65]:
patient = patient.iloc[:,5:]

In [72]:
patient.drop('Admission_Deposit', axis=1, inplace=True)

In [73]:
hospital.to_csv('/Users/skylark/Desktop/healthcare II/hospital.csv')
patient.to_csv('/Users/skylark/Desktop/healthcare II/patient.csv')

In [37]:
#testing = pd.concat([data_new.iloc[:,:-5], hospital, patient.iloc[:,4:]], axis=1)
#testing.drop(['case_id', 'patientid'], axis=1, inplace=True)

drop_cols = ['pat_visitors_code','pat_bed_grade_code','pat_age_code','pat_extra_rooms_code','pat_admission_code','pat_illness_code',
 'hosp_ward_facility','dept_ward_ward_fac','dept_ward_bed_grade']

drop_cols_2 = ['pat_visitors_code','admission_ward_fac_count', 'illness_ward_fac_count', 'pat_age_mean_deposit','pat_bed_grade_code','pat_age_code','pat_extra_rooms_code','pat_admission_code','pat_illness_code',
 'hosp_ward_facility','dept_ward_ward_fac','dept_ward_bed_grade','pat_id_admission_type','pat_id_illness_type','new_hosp_code','new_hosp_dept_code','dept_ward_code']

In [113]:
data_new

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit
0,1,8,2,3,2,3,3,2,5,2.0,31397,7.0,0,0,2,5,4911.0
1,2,2,2,5,2,2,3,3,5,2.0,31397,7.0,1,0,2,5,5954.0
2,3,10,4,1,0,2,1,3,4,2.0,31397,7.0,1,0,2,5,4745.0
3,4,26,1,2,1,2,3,2,3,2.0,31397,7.0,1,0,2,5,7272.0
4,5,26,1,2,1,2,3,3,3,2.0,31397,7.0,1,0,2,5,5558.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455490,455491,11,1,2,1,4,1,1,3,3.0,41160,3.0,0,1,4,4,6313.0
455491,455492,25,4,1,0,2,3,2,4,4.0,30985,7.0,0,2,2,0,3510.0
455492,455493,30,2,3,2,2,1,2,0,4.0,81811,12.0,2,1,2,0,7190.0
455493,455494,5,0,1,0,2,1,2,4,4.0,57021,10.0,1,1,2,4,5435.0


#### Best result at n_estimators = 246

#### Best result -> drop_cols with n_estimators = 500

#### Best result -> drop_cols_2 with n_estimators = 175

#### best result -> n_estimators = 253

### LGBM

In [109]:
train_x, test_x, train_y, test_y = train_test_split(testing[:len(train)], target, test_size=0.3, random_state=38)
lgbm = LGBMClassifier(n_estimators=253)
lgbm.fit(train_x, train_y)

lgbm.score(train_x, train_y)

0.4899419486240837

In [110]:
lgbm.score(test_x, test_y)

0.43584348699912073

In [111]:
pred = lgbm.predict(testing[len(train):])
sample['Stay'] = pred
sample['Stay'] = sample['Stay'].map(lambda x: rev_stay_dict[x])
sample.set_index('case_id').to_csv('/Users/skylark/Desktop/healthcare II/result.csv')

In [80]:
imp=pd.DataFrame({'cols':train_x.columns, 'imp':lgbm.feature_importances_})

In [81]:
list(imp[imp.imp==0].cols)

[]